In [2]:
import os
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, UpSampling2D

# Define the paths to the clear and foggy image folders
clear_folder = "C:/Users/HP/Downloads/frida2/frida2/"
foggy_homogeneous_folder = "C:/Users/HP/Downloads/frida2/frida2/"
foggy_heterogeneous_folder = "C:/Users/HP/Downloads/frida2/frida2/"
foggy_cloudy_homogeneous_folder = "C:/Users/HP/Downloads/frida2/frida2/"
foggy_cloudy_heterogeneous_folder = "C:/Users/HP/Downloads/frida2/frida2/"

# Set the desired image size
image_height = 64
image_width = 64
num_channels = 3

# Load the clear and foggy images
clear_images = []
foggy_images = []

for filename in os.listdir(clear_folder):
    if filename.endswith(".png"):
        clear_img = cv2.imread(os.path.join(clear_folder, filename))
        foggy_homogeneous_filenames = [filename.replace("LIma", "U080")] * 4  # Replace clear image name with corresponding homogeneous foggy image names
        foggy_heterogeneous_filenames = [filename.replace("LIma", "K080")] * 4  # Replace clear image name with corresponding heterogeneous foggy image names
        foggy_cloudy_homogeneous_filenames = [filename.replace("LIma", "L080")] * 4  # Replace clear image name with corresponding cloudy homogeneous foggy image names
        foggy_cloudy_heterogeneous_filenames = [filename.replace("LIma", "M080")] * 4  # Replace clear image name with corresponding cloudy heterogeneous foggy image names
        
        foggy_homogeneous_imgs = []
        foggy_heterogeneous_imgs = []
        foggy_cloudy_homogeneous_imgs = []
        foggy_cloudy_heterogeneous_imgs = []
        
        for f_h in foggy_homogeneous_filenames:
            foggy_homogeneous_imgs.append(cv2.imread(os.path.join(foggy_homogeneous_folder, f_h)))
        for f_h in foggy_heterogeneous_filenames:
            foggy_heterogeneous_imgs.append(cv2.imread(os.path.join(foggy_heterogeneous_folder, f_h)))
        for f_h in foggy_cloudy_homogeneous_filenames:
            foggy_cloudy_homogeneous_imgs.append(cv2.imread(os.path.join(foggy_cloudy_homogeneous_folder, f_h)))
        for f_h in foggy_cloudy_heterogeneous_filenames:
            foggy_cloudy_heterogeneous_imgs.append(cv2.imread(os.path.join(foggy_cloudy_heterogeneous_folder, f_h)))
        
        clear_img = cv2.resize(clear_img, (image_width, image_height))
        foggy_homogeneous_imgs = [cv2.resize(img, (image_width, image_height)) for img in foggy_homogeneous_imgs]
        foggy_heterogeneous_imgs = [cv2.resize(img, (image_width, image_height)) for img in foggy_heterogeneous_imgs]
        foggy_cloudy_homogeneous_imgs = [cv2.resize(img, (image_width, image_height)) for img in foggy_cloudy_homogeneous_imgs]
        foggy_cloudy_heterogeneous_imgs = [cv2.resize(img, (image_width, image_height)) for img in foggy_cloudy_heterogeneous_imgs]
        
        clear_images.append(clear_img)
        foggy_images.extend(foggy_homogeneous_imgs)
        foggy_images.extend(foggy_heterogeneous_imgs)
        foggy_images.extend(foggy_cloudy_homogeneous_imgs)
        foggy_images.extend(foggy_cloudy_heterogeneous_imgs)

# Convert the lists of images to numpy arrays
clear_images = np.array(clear_images)
foggy_images = np.array(foggy_images)

# Normalize pixel values to range between 0 and 1
clear_images = clear_images.astype('float32') / 255.0
foggy_images = foggy_images.astype('float32') / 255.0

# Define the CNN model architecture
model = Sequential()
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', input_shape=(image_height, image_width, num_channels)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(num_channels, (3, 3), activation='relu', padding='same'))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(foggy_images, clear_images, epochs=5, batch_size=4)

# Save the trained model
model.save("foggy_to_clear_model.h5")


ValueError: Data cardinality is ambiguous:
  x sizes: 5280
  y sizes: 330
Make sure all arrays contain the same number of samples.

import os

source_folder = "C:\Users\HP\Documents\Semester (summer)\Aim Lab Internship\Multi fog\U"  # Folder containing the images
target_folder = "converted"  # Folder to save the converted images

# Create the target folder if it doesn't exist
if not os.path.exists(target_folder):
    os.makedirs(target_folder)

# Iterate through the files in the source folder
for filename in os.listdir(source_folder):
    if filename.startswith("U") and filename.endswith(".png"):
        # Extract the desired part of the filename
        new_filename = filename[1:]

        # Build the source and target file paths
        source_path = os.path.join(source_folder, filename)
        target_path = os.path.join(target_folder, new_filename)

        # Rename and move the file
        os.rename(source_path, target_path)


In [ ]:
import cv2
import numpy as np
from keras.models import load_model

# Set the desired image size
image_height = 64
image_width = 64
num_channels = 3

# Load the trained model
model = load_model("foggy_to_clear_model.h5")

# Load a foggy image for testing
foggy_image_path = "C:/Users/HP/Documents/Semester (summer)/Aim Lab Internship/Fog/foggy-001.jpg"
foggy_image = cv2.imread(foggy_image_path)

# Resize the foggy image to the desired size
foggy_image = cv2.resize(foggy_image, (image_width, image_height))

# Preprocess the foggy image
foggy_image = foggy_image.astype('float32') / 255.0
foggy_image = np.expand_dims(foggy_image, axis=0)

# Use the model to predict the clear image
clear_image = model.predict(foggy_image)

# Rescale the pixel values to the original range
clear_image = clear_image * 255.0
clear_image = clear_image.astype('uint8')

# Reshape the clear image
clear_image = np.squeeze(clear_image, axis=0)

# Display the clear image
cv2.imshow("Clear Image", clear_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

